# Mathematical Finance — Week 5

## Asian Option Pricing via Monte-Carlo

We work in a Black–Scholes economy with risk-neutral dynamics

$$
dS_t = r S_t\,dt + \sigma S_t\,dW_t,
\qquad
S_0 = 100,\ r = 0.02,\ \sigma = 0.15,\ T = 10.
$$

The observation times are

$$
t_j = j\Delta t,
\qquad
\Delta t = \frac{T}{10} = 1,
\qquad
j = 1,\dots,10.
$$

A single simulation path uses the exact solution of geometric Brownian motion:

$$
S_{t_j}
= S_{t_{j-1}} \exp\!\left( (r - \tfrac12\sigma^2)\Delta t + \sigma\sqrt{\Delta t}\, Z_j \right),
\qquad
Z_j \sim N(0,1)\ \text{i.i.d.}
$$

For one path the arithmetic average is

$$
A = \frac{1}{10}\sum_{j=1}^{10} S_{t_j}.
$$

The payoff of the Asian fixed-strike call is

$$
f(S_1,\dots,S_{10})
= \max(A - 100,\ 0).
$$

The risk-neutral price of the option is

$$
V = e^{-rT}\,\mathbb{E}[\,f(S_1,\dots,S_{10})\,].
$$

Using Monte-Carlo with $M$ simulated paths, define for each path $m$:

$$
A^{(m)} = \frac{1}{10}\sum_{j=1}^{10} S_{t_j}^{(m)},
\qquad
Y_m = \max(A^{(m)} - 100,\ 0).
$$

The Monte-Carlo price estimator is

$$
\widehat{V}_{MC}
= e^{-rT}\,\frac{1}{M}\sum_{m=1}^{M} Y_m .
$$

Let

$$
\bar{Y} = \frac{1}{M}\sum_{m=1}^{M} Y_m,
\qquad
\widehat{\sigma}_Y^2
= \frac{1}{M-1}\sum_{m=1}^{M}(Y_m - \bar{Y})^2.
$$

Then the standard error of the Monte-Carlo estimator is

$$
\mathrm{SE}(\widehat{V}_{MC})
= e^{-rT}\,\frac{\widehat{\sigma}_Y}{\sqrt{M}}.
$$

---

## Result

With $M = 200{,}000$ simulated paths:

$$
\widehat{V}_{MC} = 15.8336, \qquad \mathrm{SE} = 0.0511.
$$

---

## Comparison to European Call

The Black–Scholes price of a European call with the same parameters $(T = 10, K = 100)$:

$$
C_{\text{European}} = 27.5713.
$$

The Asian option is **74.13% cheaper** than the European option.

This makes sense because the Asian option's payoff depends on the arithmetic average of prices over time, which reduces volatility and therefore value compared to a European option that depends only on the terminal price.

## Figures (from Python script)

The following figures are generated by running `pricing_simulation.py` and saved under `outputs/`.

![GBM Paths](outputs/paths.png)

![Terminal Price Distribution](outputs/terminal_hist.png)

## First Moment of the Asian Average

The arithmetic average is

$$
A=\frac{1}{10}\sum_{j=1}^{10} S_j,
\qquad S_j = S_{t_j},\ t_j = j.
$$

Under the risk-neutral measure,

$$
\mathbb{E}^{\mathbb{Q}}[S_j] = S_0 e^{r t_j}
= 100\, e^{0.02 j}.
$$

Therefore,

$$
\mathbb{E}^{\mathbb{Q}}[A]
= \frac{1}{10} \sum_{j=1}^{10} \mathbb{E}^{\mathbb{Q}}[S_j]
= \frac{1}{10} \sum_{j=1}^{10} 100\, e^{0.02 j}.
$$

Compute the numerical value:

$$
\mathbb{E}^{\mathbb{Q}}[A]
= \frac{100}{10} \sum_{j=1}^{10} e^{0.02 j}
= 10 \sum_{j=1}^{10} e^{0.02 j}.
$$

Using the exact exponential values:

$$
\sum_{j=1}^{10} e^{0.02 j}
= 1.02020134
+ 1.04081077
+ 1.06183655
+ 1.08328707
+ 1.10517019
+ 1.12749685
+ 1.15027380
+ 1.17351087
+ 1.19721736
+ 1.22140276
= 11.18120829.
$$

Thus,

$$
\boxed{
\mathbb{E}^{\mathbb{Q}}[A]
= 10 \times 11.18120829
= 111.8121
}.
$$

Rounded:

$$
\mathbb{E}^{\mathbb{Q}}[A] \approx 111.81.
$$

## Second Moment and Variance of the Asian Average

The second moment is given by:

$$
\mathbb{E}[A^2]
= \mathbb{E}\!\left[\left(\frac{1}{10}\sum_{i=1}^{10} S_i\right)^2\right]
= \frac{1}{100}\,\mathbb{E}\!\left[\sum_{i=1}^{10}\sum_{j=1}^{10} S_i S_j\right].
$$

Under the risk-neutral GBM model:

$$
\mathbb{E}[S_i S_j]
= S_0^2 \exp\!\big(r(t_i + t_j) + \sigma^2 \min(t_i, t_j)\big).
$$

Therefore:

$$
\mathbb{E}[A^2]
= \frac{1}{100}\sum_{i=1}^{10}\sum_{j=1}^{10} S_0^2 \exp\!\big(r(i + j) + \sigma^2 \min(i,j)\big).
$$

The variance is:

$$
\mathrm{Var}(A) = \mathbb{E}[A^2] - \big(\mathbb{E}[A]\big)^2.
$$

---

### Numerical Results

With $S_0 = 100$, $r = 0.02$, $\sigma = 0.15$, $n = 10$:

$$
\begin{align*}
\mathbb{E}[A] &= 111.8121, \\
\mathbb{E}[A^2] &= 13{,}704.6832, \\
\mathrm{Var}(A) &= 1{,}202.7413, \\
\mathrm{Std}(A) &= 34.6806.
\end{align*}
$$

## Log-Normal Approximation of A

We construct an approximating log-normal random variable:

$$
\tilde{A} = A_0 e^{(r - \frac{1}{2}b^2)T + bW_T}
$$

by choosing $A_0$, $b$ such that the first and second moments of $\tilde{A}$ match those of $A$.

Under the risk-neutral measure:

$$
\mathbb{E}^{\mathbb{Q}}[\tilde{A}] = \mathbb{E}^{\mathbb{Q}}\!\left[A_0 e^{(r - \frac{1}{2}b^2)T + bW_T}\right] = A_0 e^{rT}
$$

$$
\mathbb{E}^{\mathbb{Q}}[\tilde{A}^2] = \mathbb{E}^{\mathbb{Q}}\!\left[A_0^2 e^{2(r - \frac{1}{2}b^2)T + 2bW_T}\right] = A_0^2 e^{(2r + b^2)T}
$$

Setting these equal to the moments of $A$:

$$
A_0 e^{rT} = \mathbb{E}[A] \quad \Rightarrow \quad A_0 = \frac{\mathbb{E}[A]}{e^{rT}}
$$

$$
A_0^2 e^{(2r + b^2)T} = \mathbb{E}[A^2] \quad \Rightarrow \quad b^2 = \frac{\ln(\mathbb{E}[A^2] / A_0^2)}{T} - 2r
$$

---

### Numerical Solution

With $\mathbb{E}[A] = 111.8121$ and $\mathbb{E}[A^2] = 13{,}704.6832$:

$$
\begin{align*}
A_0 &= \frac{111.8121}{e^{0.02 \times 10}} = 91.5440, \\
b^2 &= \frac{\ln(13{,}704.6832 / 91.5440^2)}{10} - 2(0.02) = 0.0091854, \\
b &= 0.0958.
\end{align*}
$$

**Verification:**
$$
\begin{align*}
\mathbb{E}[\tilde{A}] &= 91.5440 \times e^{0.02 \times 10} = 111.8121 \quad \checkmark \\
\mathbb{E}[\tilde{A}^2] &= 91.5440^2 \times e^{(2 \times 0.02 + 0.0091854) \times 10} = 13{,}704.6832 \quad \checkmark
\end{align*}
$$

## Closed-Form Price via Log-Normal Approximation

Compute the closed-form price for $e^{-rT} \mathbb{E}[\max(\tilde{A} - K, 0)]$ using the Black-Scholes formula with initial price $A_0$ and volatility $b$.

The log-normal call option price is:

$$
V_{\text{lognorm}} = e^{-rT} \left[ A_0 e^{rT} N(d_1) - K N(d_2) \right]
$$

where

$$
d_1 = \frac{\ln(A_0 / K) + (r + \frac{1}{2}b^2)T}{b\sqrt{T}}, \qquad d_2 = d_1 - b\sqrt{T}.
$$

---

### Monte Carlo Asian Price (500,000 paths)

$$
\hat{V}_{\text{Asian,MC}} = 15.8255
$$

Standard error: $\text{SE} = 0.0322$

95% confidence interval:

$$
\hat{V}_{\text{Asian,MC}} \pm 1.96\,\text{SE}
= 15.8255 \pm 1.96 \times 0.0322
= [15.7624,\; 15.8887].
$$

---

### Log-Normal Approximation Price

Using $A_0 = 91.5440$, $b = 0.0958$, $K=100$, $r=0.02$, $T=10$:

$$
\begin{align*}
d_1 &= 0.5199, \quad d_2 = 0.2169, \\
N(d_1) &= 0.6984, \quad N(d_2) = 0.5858,
\end{align*}
$$

$$
V_{\text{lognorm}}
= A_0 N(d_1) - K e^{-rT} N(d_2)
\approx 15.9738.
$$

---

### Comparison

- **Monte Carlo price:** $15.8255$
- **Log-normal approximation:** $15.9738$
- **Absolute difference:** $15.9738 - 15.8255 = 0.1483$
- **Relative difference:** $\approx 0.94\%$

**Conclusion:** The log-normal closed-form **slightly overprices** the Asian call and lies just above the 95% MC confidence interval, but it is still quite close (sub-1% error). The approximation provides an efficient alternative to Monte Carlo simulation with good accuracy.

## Unit-Linked Contract with Continuous Guarantee

### Interpretation

Fund $S_t$ follows Black–Scholes (risk-neutral):

$$
dS_t = r S_t\,dt + \sigma S_t\,dW_t,
\qquad
S_0 = 100,\ r=0.02,\ \sigma=0.15,\ T=10.
$$

The contract value at any time $t$ is:

* If you "keep" the contract: value of fund $S_t$ plus future guarantee.
* If you "exercise" the continuous guarantee: you walk away with **100** immediately.

So the contract is equivalent to:

$$
\text{UL with continuous guarantee}
= S_0 + \text{American put on } S \text{ with } K=100,T=10.
$$

We will price it directly in a binomial tree.

---

### Binomial Tree (CRR) for the Contract

For a tree with $N$ steps:

$$
\Delta t = \frac{T}{N},\quad
u = e^{\sigma\sqrt{\Delta t}},\quad
d = \frac{1}{u},\quad
p = \frac{e^{r\Delta t}-d}{u-d},\quad
\text{disc} = e^{-r\Delta t}.
$$

Stock prices at node $(n,j)$ (time $t_n = n\Delta t$, $j$ up-moves):

$$
S_{n,j} = S_0\,u^{j} d^{n-j},
\qquad n=0,\dots,N,\ j=0,\dots,n.
$$

Terminal contract values (at $T$):

$$
V_{N,j} = \max\{S_{N,j}, 100\}
\quad\text{for } j=0,\dots,N.
$$

Backward induction for $n=N-1,\dots,0$:

1. **Continuation value** at node $(n,j)$:

$$
V^{\text{cont}}_{n,j}
= \text{disc} \big( p\,V_{n+1,j+1} + (1-p)\,V_{n+1,j} \big).
$$

2. **Early exercise value** of the guarantee = 100.

3. Node value:

$$
V_{n,j} = \max\{V^{\text{cont}}_{n,j},\ 100\}.
$$

Initial price:

$$
V_0 = V_{0,0}.
$$

---

### Numerical Convergence

With $S_0=100, K=100, r=0.02, \sigma=0.15, T=10$:

| $N$ (steps) | UL price $V_0^{(N)}$ |
| ----------: | -------------------: |
|          10 |             111.6668 |
|          20 |             111.7684 |
|          40 |             111.8094 |
|          80 |             111.8390 |
|         160 |             111.8529 |
|         320 |             111.8597 |
|         640 |             111.8630 |
|        1280 |             111.8645 |

So the binomial price converges to roughly

$$
\boxed{V_0 \approx 111.86}.
$$

![Convergence Plot](outputs/ul_continuous_convergence.png)

## LSMC Pricing of Unit-Linked Contract

We price the **unit-linked with continuous guarantee 100** using Least-Squares Monte Carlo (LSMC).

The contract value equals

$$
V_0 = S_0 + P_0^{\text{Am.put}}(K=100,T=10),
$$

so we only need the price of an **American put** on $S_t$ with strike $K=100$.

Under the risk-neutral Black–Scholes model,

$$
dS_t = r S_t\,dt + \sigma S_t\,dW_t,
\qquad
S_0 = 100,\ r=0.02,\ \sigma=0.15,\ T=10.
$$

We allow exercise at discrete times

$$
t_k = k\Delta t,\quad k=1,\dots,N,\quad \Delta t = \frac{T}{N}.
$$

The immediate exercise payoff of the put at time $t_k$ is

$$
h_k(S_{t_k}) = \max(K - S_{t_k},0).
$$

The true price is

$$
P_0^{\text{Am}} = \sup_{\tau \in \mathcal{T}}
\mathbb{E}^{\mathbb{Q}}\!\big[e^{-r\tau} h_\tau(S_\tau)\big],
$$

where $\mathcal{T}$ is the set of stopping times taking values in $\{t_1,\dots,t_N\}$.

---

### Least–Squares Monte Carlo (LSMC) Algorithm

1. Simulate $M$ paths of $S_t$ at the times $t_k$:

$$
S_{t_{k+1}}^{(m)}
= S_{t_k}^{(m)}
\exp\!\big((r-\tfrac12\sigma^2)\Delta t + \sigma\sqrt{\Delta t}\,Z_k^{(m)}\big),
\quad Z_k^{(m)}\sim N(0,1).
$$

2. At maturity $t_N$ set cashflows

$$
C_N^{(m)} = h_N\big(S_{t_N}^{(m)}\big).
$$

3. For $k = N-1,\dots,1$ (backward):

   * For each path $m$, compute immediate exercise value
     $h_k^{(m)} = h_k(S_{t_k}^{(m)})$.

   * Consider only **in-the-money** paths where $h_k^{(m)} > 0$.
     On those paths regress the discounted future cashflow
     $Y^{(m)} = e^{-r\Delta t} C_{k+1}^{(m)}$ on a set of basis functions
     $\phi_1,\dots,\phi_p$ of the state $S_{t_k}^{(m)}$:

$$
Y^{(m)} \approx
\sum_{j=1}^{p} \beta_j \phi_j\big(S_{t_k}^{(m)}\big).
$$

     In matrix form, $\beta$ is obtained by least squares.

   * The **estimated continuation value** on each path is

$$
\widehat{CV}_k^{(m)} =
\sum_{j=1}^{p} \beta_j \phi_j\big(S_{t_k}^{(m)}\big).
$$

   * Exercise rule:

$$
C_k^{(m)} =
\begin{cases}
h_k^{(m)}, & h_k^{(m)} > \widehat{CV}_k^{(m)},\\[4pt]
e^{-r\Delta t} C_{k+1}^{(m)}, & \text{otherwise}.
\end{cases}
$$

4. After stepping back to $k=1$, the time-0 American put price is

$$
\widehat{P}_0^{\text{Am}}
= e^{-r\Delta t}\,\frac{1}{M}\sum_{m=1}^{M} C_1^{(m)}.
$$

The **unit-linked price** is then

$$
\widehat{V}_0^{\text{UL}} = S_0 + \widehat{P}_0^{\text{Am}}.
$$

---

### LSMC Results with Different Basis Functions

With $M = 100{,}000$ paths and $N = 50$ exercise times:

| Basis      | Am. Put Price | UL Price   | Diff vs. Binomial |
| :--------- | ------------: | ---------: | ----------------: |
| poly2      |      11.7367  |  111.7367  |       -0.115%     |
| poly3      |      11.7591  |  111.7591  |       -0.095%     |
| laguerre   |      11.7367  |  111.7367  |       -0.115%     |

**Binomial benchmark:** $V_0^{\text{UL}} \approx 111.865$ (from N=1280 tree)

---

### Discussion

* **poly2:** Basis of $\{1, S, S^2\}$ gives good approximation
* **poly3:** Adding $S^3$ term slightly changes the estimate
* **laguerre:** Laguerre polynomials $\{L_0, L_1, L_2\}$ also work well
* All LSMC estimates are within ~0.1% of the binomial benchmark
* Choice of basis affects bias/variance of the continuation regression

The LSMC method provides a flexible alternative to binomial trees for pricing American options, especially in higher dimensions where trees become computationally prohibitive.

## Comparison: Final Guarantee vs. Continuous Guarantee

We now compare two unit-linked contracts:

1. **Final guarantee only** (maturity guarantee 100)
2. **Continuous guarantee** (American-style, can exercise anytime)

---

### Unit-Linked with Final Guarantee Only

This is $S_0$ plus a **European put** with $K=100, T=10$:

Using put-call parity:

$$
P_{\text{Eur}} = C_{\text{BS}} - S_0 + K e^{-rT},
$$

With $C_{\text{BS}} = 27.571349$ (European call), $S_0=K=100$, $r=0.02$, $T=10$:

$$
P_{\text{Eur}} \approx 27.571349 - 100 + 100 e^{-0.2}
\approx 9.4444,
$$

$$
V_0^{\text{final guarantee}} = S_0 + P_{\text{Eur}} \approx 109.44.
$$

---

### Unit-Linked with Continuous Guarantee

From the binomial tree and LSMC methods:

$$
V_0^{\text{continuous}} \approx 111.87 \quad \text{(binomial N=1280)}.
$$

---

### Comparison

$$
V_0^{\text{continuous}} - V_0^{\text{final}}
\approx 111.87 - 109.44 \approx 2.42,
$$

The continuous-guarantee UL is about **2.2% more expensive** than the final-guarantee UL.

---

### Early Exercise Premium

The American put value is:

$$
P_{\text{Am}} = V_0^{\text{continuous}} - S_0 \approx 11.87,
$$

compared to the European put:

$$
P_{\text{Eur}} \approx 9.44.
$$

The early-exercise premium is:

$$
P_{\text{Am}} - P_{\text{Eur}} \approx 2.42,
$$

which is about **25.6% extra** on top of the European put value.

---

### Interpretation

The premium of **2.42** reflects the value of being able to exercise the guarantee at any time during the contract life, not just at maturity. This is particularly valuable when the fund value drops significantly, allowing the policyholder to lock in the guarantee before potential recovery.